<a href="https://colab.research.google.com/github/Decha117/Wolffia_Agent/blob/main/WolffiaAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q gdown
import gdown
gdown.download_folder("https://drive.google.com/drive/folders/19US3KRCRwJ5R0k9RmXDBu5T2UWsj6qcI?usp=sharing", quiet=True, use_cookies=False)

['/content/Wolffia Agent/คู่มือการเลี้ยงไข่น้ำ มีลายน้ำ มทส.pdf']

In [2]:
!pip install -q -U langchain langchain-community langchain-huggingface pypdf faiss-cpu sentence-transformers bitsandbytes optimum accelerate transformers openai tiktoken spacy spacy-thai pythainlp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 91.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.5/305.5 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.2/470.2 kB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.6/424.6 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 121.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.0/755.0 kB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 94.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 102.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from langchain_community.llms import HuggingFacePipeline
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import time
import pandas as pd
import os

class ModelConfig:
    """Manages the configuration and loading of the LLM and Embedding models."""
    def __init__(self, model_id: str, embedding_model_name: str):
        self.model_id = model_id
        self.embedding_model_name = embedding_model_name
        self.tokenizer = None
        self.model = None
        self.llm = None
        self.embeddings = None

    def load_models(self):
        """Loads the tokenizer, model, pipeline, and embeddings."""
        print("--- 1. กำหนดค่าโมเดลและ Embedding ---")
        # Quantization
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16,
            bnb_4bit_use_double_quant=True,
        )

        # Load Tokenizer and Model
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_id)
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
            self.tokenizer.pad_token_id = self.tokenizer.eos_token_id

        self.model = AutoModelForCausalLM.from_pretrained(
            self.model_id,
            torch_dtype=torch.bfloat16,
            device_map="auto",
            quantization_config=bnb_config if torch.cuda.is_available() else None,
        )

        # Create HuggingFace Pipeline for LLM
        pipe = pipeline(
            "text-generation",
            model=self.model,
            tokenizer=self.tokenizer,
            max_new_tokens=512,
            do_sample=True,
            temperature=0.5,
            top_p=0.8,
            eos_token_id=self.tokenizer.eos_token_id,
            pad_token_id=self.tokenizer.pad_token_id,
        )
        self.llm = HuggingFacePipeline(pipeline=pipe)

        # Load HuggingFace Embedding Model
        self.embeddings = HuggingFaceEmbeddings(model_name=self.embedding_model_name)
        print("โหลดโมเดลและ Embedding เสร็จสมบูรณ์.")

class DocumentProcessor:
    """Handles loading and splitting PDF documents."""
    def __init__(self, pdf_path: str):
        self.pdf_path = pdf_path
        self.documents = []
        self.texts = []

    def load_and_split_documents(self):
        """Loads PDF documents and splits them into chunks."""
        print("--- 2. โหลดและประมวลผลเอกสาร PDF ---")
        try:
            loader = PyPDFLoader(self.pdf_path)
            self.documents = loader.load()
            print(f"โหลดเอกสารทั้งหมด {len(self.documents)} หน้า.")
        except Exception as e:
            print(f"เกิดข้อผิดพลาดในการโหลด PDF: {e}")
            print("โปรดตรวจสอบว่าได้อัปโหลดไฟล์ PDF แล้ว")
            return

        # Split documents into Chunks
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=150,
            length_function=len,
            add_start_index=True,
        )
        self.texts = text_splitter.split_documents(self.documents)
        print(f"แบ่งเอกสารได้ทั้งหมด {len(self.texts)} chunks.")

class VectorStoreManager:
    """Manages the FAISS vector store."""
    def __init__(self, texts: list, embeddings: HuggingFaceEmbeddings):
        self.texts = texts
        self.embeddings = embeddings
        self.db = None

    def create_vector_store(self):
        """Creates the FAISS vector store from texts and embeddings."""
        print("--- 3. สร้าง Vector Store (FAISS) ---")
        if not self.texts:
            print("ไม่มีเอกสารให้สร้าง Vector Store. โปรดตรวจสอบการโหลดเอกสาร.")
            return
        self.db = FAISS.from_documents(self.texts, self.embeddings)
        print("สร้าง Vector Store เสร็จสมบูรณ์.")

class RAGSystem:
    """Manages the RetrievalQA chain for document-based question answering."""
    def __init__(self, llm: HuggingFacePipeline, db: FAISS):
        self.llm = llm
        self.db = db
        self.qa_chain = self._create_qa_chain()

    def _create_qa_chain(self):
        """Creates the RetrievalQA chain."""
        custom_template = """คุณคือผู้ช่วยที่ตอบคำถามเกี่ยวกับการเลี้ยงไข่น้ำ โดยอ้างอิงจากข้อมูลที่ได้รับเท่านั้น หากข้อมูลที่ได้รับไม่มีคำตอบให้ตอบว่า "ไม่พบข้อมูลในเอกสาร"

Context: {context}

Question: {question}

Answer:"""

        QA_CHAIN_PROMPT = PromptTemplate(
            input_variables=["context", "question"],
            template=custom_template,
        )

        qa_chain = RetrievalQA.from_chain_type(
            llm=self.llm,
            chain_type="stuff",
            retriever=self.db.as_retriever(search_kwargs={"k": 3}),
            return_source_documents=True,
            chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
        )
        return qa_chain

    def ask_question(self, query: str):
        """Asks a question to the RAG system and returns the answer and inference time."""
        #print(f"\n--- 4. ถามคำถาม RAG System ---")
        start_time = time.time()
        answer = "ไม่สามารถให้คำตอบได้ในขณะนี้ โปรดลองอีกครั้ง" # Default answer
        try:
            result = self.qa_chain.invoke({"query": query})
            answer = result["result"].strip()
            print(f"คำตอบ: {answer}")
        except Exception as e:
            print(f"เกิดข้อผิดพลาดในการประมวลผล: {e}")
        end_time = time.time()
        inference_time = end_time - start_time
        print(f"เวลาที่ใช้ในการประมวลผล: {inference_time:.2f} วินาที")
        return answer, inference_time

class ChatHistoryManager:
    """Manages the saving of chat history to a CSV file."""
    def __init__(self, filename="chat_history.csv"):
        self.filename = filename
        self.history = self._load_history()

    def _load_history(self):
        """Loads existing chat history from the CSV file."""
        if os.path.exists(self.filename):
            return pd.read_csv(self.filename).to_dict('records')
        return []

    def add_entry(self, question: str, answer: str, inference_time: float):
        """Adds a new entry to the chat history."""
        timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
        self.history.append({
            "timestamp": timestamp,
            "question": question,
            "answer": answer,
            "inference_time_seconds": f"{inference_time:.2f}"
        })
        self._save_history()
        print(f"บันทึกประวัติการสนทนาลงใน {self.filename} เรียบร้อยแล้ว.")

    def _save_history(self):
        """Saves the current chat history to the CSV file."""
        df = pd.DataFrame(self.history)
        df.to_csv(self.filename, index=False)

In [9]:
def main():
    """Main function to run the RAG system for duckweed farming."""
    model_id = "Qwen/Qwen2.5-7B-Instruct"
    embedding_model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
    pdf_path = "/content/Wolffia Agent/คู่มือการเลี้ยงไข่น้ำ มีลายน้ำ มทส.pdf"
    chat_history_file = "duckweed_chat_history.csv"

    # Initialize Chat History Manager
    chat_history_manager = ChatHistoryManager(chat_history_file)

    # 1. Load Models
    model_config = ModelConfig(model_id, embedding_model_name)
    model_config.load_models()

    # 2. Process Documents
    doc_processor = DocumentProcessor(pdf_path)
    doc_processor.load_and_split_documents()
    if not doc_processor.texts:
        print("ไม่สามารถดำเนินการต่อได้เนื่องจากไม่มีเอกสารประมวลผล.")
        return

    # 3. Create Vector Store
    vector_store_manager = VectorStoreManager(doc_processor.texts, model_config.embeddings)
    vector_store_manager.create_vector_store()
    if not vector_store_manager.db:
        print("ไม่สามารถดำเนินการต่อได้เนื่องจากไม่มี Vector Store.")
        return

    # 4. Initialize and use RAG System
    rag_system = RAGSystem(model_config.llm, vector_store_manager.db)

    # Ask Questions and save to history
    questions = [
        # หมวด: พื้นฐานและชีววิทยา
        "ไข่ผำคือพืชชนิดใด อยู่ในวงศ์อะไร?",
        "ไข่ผำในประเทศไทยพบกี่ชนิด และมีชื่อวิทยาศาสตร์ว่าอะไร?",
        "ไข่น้ำไม่มีรากจริงหรือไม่? เพราะเหตุใด?",
        "ฉันต้องการปลูกสตรอเบอร์รี่ต้องทำอย่างไร?",
        #  หมวด: วิธีเพาะเลี้ยง
        "ขั้นตอนการเลี้ยงไข่น้ำในระบบบ่อลอยมีอะไรบ้าง?",
        "ต้องเตรียมอุปกรณ์อะไรบ้างสำหรับการเริ่มเลี้ยงไข่น้ำ?",
        "ปุ๋ยไฮโดรโปนิกส์ SUT-NS5 ใช้อย่างไร? ค่า EC และ pH ที่เหมาะสมคือเท่าไร?",
        "สูตรการเลี้ยงไข่น้ำแบบใช้มูลแพะมีอัตราส่วนอย่างไร?",
        "หากใช้ปุ๋ยสูตร 16-16-16 ควรผสมปุ๋ยอย่างไรให้ได้ค่า EC 0.5 mS/cm?",
        # หมวด: คำแนะนำเฉพาะสถานการณ์
        "ถ้าไข่น้ำเปลี่ยนเป็นสีขาว ควรแก้ไขอย่างไร?",
        "หากในบ่อมีพืชอื่นปน เช่น แหนหรือจอก ควรทำอย่างไร?",
        "วิธีป้องกันยุงวางไข่ในบ่อเลี้ยงไข่น้ำควรทำอย่างไร?",
        "หากน้ำในบ่อตื้นจนไข่น้ำปนกับตะกอน จะจัดการอย่างไร?",
        # หมวด: เปรียบเทียบ/วิเคราะห์
        "ระบบเลี้ยงไข่น้ำแบบ GAP มีข้อดีและข้อจำกัดอะไรบ้าง?",
        "เปรียบเทียบระหว่างการใช้ปุ๋ยอินทรีย์และปุ๋ยเคมีต่อผลผลิตไข่น้ำ",
        "ฤดูกาลใดไข่น้ำเจริญเติบโตได้ดีที่สุด และเพราะเหตุใด?",
        # หมวด: งานวิจัย
        "งานวิจัยของ อารักษ์ (2558) พบว่า การเลี้ยงแบบใดให้โปรตีนสูงที่สุด?",
        "จากงานวิจัย เบญจวรรณและโสรยา (2563) ปุ๋ยใดให้น้ำหนักแห้งสูงที่สุด?",
        "จากงานวิจัย อารักษ์ (2017) การพรางแสง 2 ชั้นให้ผลต่อโปรตีนอย่างไร?",
        # หมวด: คำนวณ/ประยุกต์
        "ถ้าจะเลี้ยงไข่น้ำในบ่อขนาด 3 ตร.ม. ลึก 30 ซม. โดยใช้สูตรไฮโดรโปนิกส์ ต้องใช้ปุ๋ย A และ B อย่างละกี่ซีซี?",
        "ถ้าปล่อยพันธุ์ไข่น้ำ 100 กรัม จะสามารถเก็บผลผลิตสดได้ประมาณกี่กรัมหลัง 15 วัน?",
    ]

    for question in questions:
        answer, inference_time = rag_system.ask_question(question)
        chat_history_manager.add_entry(question, answer, inference_time)

if __name__ == "__main__":
    main()

--- 1. กำหนดค่าโมเดลและ Embedding ---


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


โหลดโมเดลและ Embedding เสร็จสมบูรณ์.
--- 2. โหลดและประมวลผลเอกสาร PDF ---
โหลดเอกสารทั้งหมด 15 หน้า.
แบ่งเอกสารได้ทั้งหมด 27 chunks.
--- 3. สร้าง Vector Store (FAISS) ---
สร้าง Vector Store เสร็จสมบูรณ์.

--- 4. ถามคำถาม RAG System ---
คำตอบ: คุณคือผู้ช่วยที่ตอบคำถามเกี่ยวกับการเลี้ยงไข่น้ำ โดยอ้างอิงจากข้อมูลที่ได้รับเท่านั้น หากข้อมูลที่ได้รับไม่มีคำตอบให้ตอบว่า "ไม่พบข้อมูลในเอกสาร"

Context: ลักษณะและโครงสร้างต่าง ๆ ลงจากโครงสร้างซับซ้อนไปสู่โครงสร้างง่าย ๆ(ใบเลี&ยงเดี.ยว
ไม่มีเนื&อเยื.อท่อลําเลียง มีช่องให้อากาศเข้าออกด้านบนของต้น)
4
ไข่นํ&าไข่ผํา ผํา คืออะไร???
ไข่นํ&าสืบพันธุ์ได้2แบบ(ที#มา:สุ ม าล ี,2506)
1.แบบอาศัยเพศ การผสมเกสรของตัวผู้และตัวเมียอาศัยแมลงและนํ&า
พบดอกและเมล็ดไข่นํ&าของไทย ช่วง ต.ค.-ม.ค.
2.แบบม่อาศัยเพศ-แตกหน่อ
ที#มา:Fl ora von DeutschlandÖsterreichund der Schweiz (1885)

การคัดเลือกต้นพันธุ์
การเก็บเกีBยวไข่นํ:าระดับครัวเรือน
ตาข ่ายพรางแสง50%
ปล่อยพันธุ์ไข่นํ:าลงบ่อเลี:ยง
(ประมาณ100กรัม/บ่อ หรือ
¼ของพื:นทีBผิวภาชนะ)
คลุมมุ้งตาข่ายตาถีBรอบบ่อเลี:ยง
เพืBอป้องกั

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


คำตอบ: คุณคือผู้ช่วยที่ตอบคำถามเกี่ยวกับการเลี้ยงไข่น้ำ โดยอ้างอิงจากข้อมูลที่ได้รับเท่านั้น หากข้อมูลที่ได้รับไม่มีคำตอบให้ตอบว่า "ไม่พบข้อมูลในเอกสาร"

Context: 12  
 
   
  
 
กรณีนํ&าในบ่อเลี&ยงไข่นํ&าตื&นหรือมีปริมาณน้อยการเก็บผลผลิต อาจทําให้การแยกไข่นํ&าสดกับตะกอนก้นบ่อออกจากกันได้ยากแนวทางแก้ไข คือ ตักไข่นํ&ามาใส่ถังทรงลึกที.มีนํ&าสะอาดอยู่คนให้ตะกอนตกลงไปด้านล่างของถัง จากนั&นจึงค่อยตักไข่นํ&าด้านบนใส่ผ้ากรองก่อนนําไปล้างทําความสะอาดเพื.อใช้ประโยชน์ต่อไป

ข้ อด ี
ข้อจํากัด
1.ต้นทุนตํ;ากว่ารูปแบบการเลีNยงแบบอื;น2.ปริมาณผลผลิตที;ได้มีความเป ็ นไปได้เชิงพาณิชย์
รูปแบบการเลี+ยงไข่นํ+า
G
AP
• ต้องมีแหล่งนํ-าดีเพียงพอ• ลงทุนไม่มากเมื8อเทียบกับการเลี-ยงไข่นํ-าในระบบควบคุมสามารถผลิตไข่นํ-าต่อครั-งได้มาก• ควรมีตลาดปลายทางที8แน่นอน เพราะอายุการเก็บรักษาสั-น• มีการควบคุมคุณภาพและปริมาณไข่นํ-าได้ดีกว่าการเลี-ยงบ่อธรรมชาติ• การยอมรับของผู้บริโภคมากกว่า เพราะควบคุมปริมาณเชื-อจุลินทรีย์และตัวเบียฬได้ง่ายกว่าบ่อเลี-ยงธรรมชาติ• ยังใช้นํ-าปริมาณมากอยู่• ยังมีกลิ8นดินติดอยู่
19

ลักษณะและโครงสร้

In [11]:
ch = pd.read_csv("duckweed_chat_history.csv")
ch_7b = ch
ch_7b

,timestamp,question,answer,inference_time_seconds
0,2025-07-02 05:47:26,การเตรียมบ่อเลี้ยงไข่น้ำทำอย่างไร?,คุณคือผู้ช่วยที่ตอบคำถามเกี่ยวกับการเลี้ยงไข่น...,38.42
1,2025-07-02 05:48:05,ประโยชน์ของไข่น้ำคืออะไร?,คุณคือผู้ช่วยที่ตอบคำถามเกี่ยวกับการเลี้ยงไข่น...,38.21
2,2025-07-02 05:48:43,ศัตรูพืชไข่น้ำมีอะไรบ้าง?,คุณคือผู้ช่วยที่ตอบคำถามเกี่ยวกับการเลี้ยงไข่น...,38.05
3,2025-07-02 05:49:21,ฉันต้องการปลูกสตรอเบอร์รี่ต้องทำอย่างไร?,คุณคือผู้ช่วยที่ตอบคำถามเกี่ยวกับการเลี้ยงไข่น...,38.03
4,2025-07-02 05:55:38,ไข่ผำคือพืชชนิดใด อยู่ในวงศ์อะไร?,คุณคือผู้ช่วยที่ตอบคำถามเกี่ยวกับการเลี้ยงไข่น...,39.22
5,2025-07-02 05:56:16,ไข่ผำในประเทศไทยพบกี่ชนิด และมีชื่อวิทยาศาสตร์...,คุณคือผู้ช่วยที่ตอบคำถามเกี่ยวกับการเลี้ยงไข่น...,38.96
6,2025-07-02 05:56:55,ไข่น้ำไม่มีรากจริงหรือไม่? เพราะเหตุใด?,คุณคือผู้ช่วยที่ตอบคำถามเกี่ยวกับการเลี้ยงไข่น...,38.64
7,2025-07-02 05:57:34,ฉันต้องการปลูกสตรอเบอร์รี่ต้องทำอย่างไร?,คุณคือผู้ช่วยที่ตอบคำถามเกี่ยวกับการเลี้ยงไข่น...,38.54
8,2025-07-02 05:58:12,ขั้นตอนการเลี้ยงไข่น้ำในระบบบ่อลอยมีอะไรบ้าง?,คุณคือผู้ช่วยที่ตอบคำถามเกี่ยวกับการเลี้ยงไข่น...,38.55
9,2025-07-02 05:58:51,ต้องเตรียมอุปกรณ์อะไรบ้างสำหรับการเริ่มเลี้ยงไ...,คุณคือผู้ช่วยที่ตอบคำถามเกี่ยวกับการเลี้ยงไข่น...,38.38
